In [2]:
import pyspark
from pandas import DataFrame, Series
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import col

sparkcontext = pyspark.SparkContext.getOrCreate(
    conf=(pyspark.SparkConf().setAppName("Dibimbing"))
)
sparkcontext.setLogLevel("WARN")

spark = pyspark.sql.SparkSession(sparkcontext.getOrCreate())

In [3]:
retail_data = spark.read.csv('data/online-retail-dataset.csv', header=True)
retail_data.show()
retail_data.printSchema()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [4]:
retail_data = retail_data.withColumn("InvoiceNo", col("InvoiceNo").cast("int"))
retail_data = retail_data.withColumn('InvoiceDate', F.to_timestamp('InvoiceDate', format='d/M/yyyy H:mm'))
retail_data = retail_data.withColumn("Quantity", col("Quantity").cast("int"))
retail_data = retail_data.withColumn("UnitPrice", col("UnitPrice").cast("decimal(10,2)"))
retail_data = retail_data.withColumn("CustomerID", col("CustomerID").cast("int"))

retail_data.printSchema()
retail_data.show()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: decimal(10,2) (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-01-12 08:26:00|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-01-12 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-01-12 08:26:00|     2.75|     17850|United Kingdom|
|  

Penjualan Terbanyak

In [5]:
retail_data.createOrReplaceTempView("revenue_data")
# Menghitung revenue per bulan dengan SQL
revenue_per_month = spark.sql("""
    SELECT
        DATE_FORMAT(InvoiceDate, 'yyyy-MM') AS Month,
        SUM(Quantity * UnitPrice) AS total_revenue
    FROM
        revenue_data
    WHERE
        InvoiceDate IS NOT NULL
    GROUP BY
        DATE_FORMAT(InvoiceDate, 'yyyy-MM')
    ORDER BY
        Month
""")

# Menampilkan hasil perhitungan
revenue_per_month.show()

+-------+-------------+
|  Month|total_revenue|
+-------+-------------+
|2010-01|     58635.56|
|2010-02|     46207.28|
|2010-03|     45620.46|
|2010-05|     31383.95|
|2010-06|     53860.18|
|2010-07|     45059.05|
|2010-08|     44189.84|
|2010-09|     52532.13|
|2010-10|     57404.91|
|2010-12|     17240.92|
|2011-01|    105452.57|
|2011-02|     71846.91|
|2011-03|    102434.16|
|2011-04|    111954.84|
|2011-05|     81058.60|
|2011-06|    120231.40|
|2011-07|    103612.42|
|2011-08|     87853.67|
|2011-09|     80370.93|
|2011-10|    118145.07|
+-------+-------------+
only showing top 20 rows



Ranking produk yang terjual

In [11]:
rank_product = spark.sql("""
    SELECT
        YEAR(InvoiceDate) AS year,
        MONTH(InvoiceDate) AS month,
        Description,
        SUM(Quantity * UnitPrice) AS total_revenue,
        RANK() OVER (PARTITION BY YEAR(InvoiceDate), MONTH(InvoiceDate) ORDER BY SUM(Quantity * UnitPrice) DESC) AS ranking
    FROM
        revenue_data
    WHERE
        InvoiceDate IS NOT NULL
    GROUP BY
        YEAR(InvoiceDate),
        MONTH(InvoiceDate),
        Description
""")
# Menampilkan hasil perhitungan
rank_product.show()

+----+-----+--------------------+-------------+-------+
|year|month|         Description|total_revenue|ranking|
+----+-----+--------------------+-------------+-------+
|2010|    1|BLACK RECORD COVE...|      1830.60|      1|
|2010|    1|RED WOOLLY HOTTIE...|      1670.55|      2|
|2010|    1|REGENCY CAKESTAND...|      1434.84|      3|
|2010|    1|WHITE HANGING HEA...|      1224.18|      4|
|2010|    1|      DOTCOM POSTAGE|      1177.26|      5|
|2010|    1|       CHILLI LIGHTS|      1109.55|      6|
|2010|    1|JUMBO BAG RED RET...|       938.72|      7|
|2010|    1|PAPER CHAIN KIT 5...|       891.10|      8|
|2010|    1|SET OF 3 COLOURED...|       809.25|      9|
|2010|    1|FAIRY TALE COTTAG...|       684.90|     10|
|2010|    1|WOODEN OWLS LIGHT...|       681.04|     11|
|2010|    1|LIGHT GARLAND BUT...|       681.04|     11|
|2010|    1|STRAWBERRY CERAMI...|       656.55|     13|
|2010|    1|RED TOADSTOOL LED...|       634.41|     14|
|2010|    1|HAND WARMER SCOTT...|       614.48| 